In [318]:
import torch
print(torch.__version__)
import sys
print(sys.version)

1.9.1.post3
3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]


In [349]:
def subidx(n:int, remove:int) -> list:
    out = list(range(n))
    out.remove(remove)
    return out

def submatrix(A:torch.tensor, j:int) -> torch.tensor:
    n = A.shape[0]
    s = subidx(n, j)
    o = A[s, :]
    o = o[:, s]
    return o

def get_eigenvector_val_old(hermitian_matrix, i, j, disable_complex=False):
    # Old way
    lam, v = torch.linalg.eig(hermitian_matrix)
    if disable_complex:
        old_eigenvector_ij = torch.abs(v[j,i]**2)
    else:
        old_eigenvector_ij = v[j,i]**2
    
    return old_eigenvector_ij

def get_eigenvector_val(hermitian_matrix, i, j, disable_complex=False):
    # Only need to calculate eigenvalues
    lam  = torch.linalg.eigvals(hermitian_matrix)
    
    n = len(lam)
    M = submatrix(hermitian_matrix, j)
    # Only need to calculate eigenvalues
    lam_submatrix = torch.linalg.eigvals(M)

    # Left side of equation 2
    left = torch.prod(torch.tensor([lam[i] - lam[k] for k in range(n) if k!=i]))
    # Right side of equation 2
    right = torch.prod(torch.tensor([lam[i] - lam_submatrix[k] for k in range(n-1)]))
    # Right divided by left
    eigenvector_ij = right / left
    
    if disable_complex:
        eigenvector_ij = torch.abs(eigenvector_ij)
    
    return eigenvector_ij

In [364]:
# Random square matrix
rand_square_matrix = torch.rand(50, 50, dtype=float)

# Hermitian matrix
hermitian_matrix = rand_square_matrix * rand_square_matrix.T

# Old 
old_eigenvector_ij = get_eigenvector_val_old(hermitian_matrix, i=0, j=0)

# New
new_eigenvector_ij = get_eigenvector_val(hermitian_matrix, i=0, j=0)

print(f'Old Method Eigenvector ij: {old_eigenvector_ij}')
print('-'*50)
print(f'New Method Eigenvector ij: {new_eigenvector_ij}')

Old Method Eigenvector ij: (0.024632040294863846-0j)
--------------------------------------------------
New Method Eigenvector ij: (0.024632040294865445+0j)


In [365]:
lam, v = torch.linalg.eig(hermitian_matrix)

In [368]:
v[0, 0] ** 2

tensor(0.0246+-0.j, dtype=torch.complex128)

In [363]:
v[0,0]**2

tensor(0.0182+-0.j, dtype=torch.complex128)

In [369]:
 # Only need to calculate eigenvalues
lam  = torch.linalg.eigvals(hermitian_matrix)

n = len(lam)
M = submatrix(hermitian_matrix, j)
# Only need to calculate eigenvalues
lam_submatrix = torch.linalg.eigvals(M)

# Left side of equation 2
left = torch.prod(torch.tensor([lam[i] - lam[k] for k in range(n) if k!=i]))
# Right side of equation 2
right = torch.prod(torch.tensor([lam[i] - lam_submatrix[k] for k in range(n-1)]))
# Right divided by left
eigenvector_ij = right / left

In [370]:

left

tensor(-7.5864e+19+0.j, dtype=torch.complex128)

In [371]:
right

tensor(-9.6471e+17+0.j, dtype=torch.complex128)

In [372]:
import numpy as np
from pprint import pprint


def cal_eigenvalues_and_eigenvectors(A):
    """
    :param A:  n x n Hermitian matrix
    :return:
    """
    eigenvalues, normed_eigenvectors = np.linalg.eig(A)
    # Below two steps are redounding for readability
    lmd = eigenvalues
    v = normed_eigenvectors
    return lmd, v


def cal_determinant(M):
    return np.linalg.det(M)


def check_lemma2():
    """
    lmd: short for lambda, i.e., eigenvalues.
         "lambda" is not a good choice in python so I use lmd instead
    v  : normed_eigenvectors
    :return:
    """
    n = np.random.randint(low=3, high=10)  # Dimension of a Hermitian matrix
    C = np.matrix(np.random.rand(n, n))              # Seed Matrix
    A = (C.getH() + C)                         # Construct Hermitian matrix
    pprint("Pick a {} x {} matrix".format(n, n))
    pprint(A)

    lmd, v = cal_eigenvalues_and_eigenvectors(A)
    pprint("Lambda Shape : {}".format(lmd.shape))
    pprint("V Shape: {}".format(v.shape))

    # Now pick a dimension: i
    i = np.random.randint(low=1, high=n)
    pprint("Pick one dimension to check : {}".format(i))

    # Now pick a dimension: j
    j = np.random.randint(low=0, high=n)
    pprint("Pick one dimension to delete : {}".format(j))

    # Now, let's compute left side of equation (2) in paper
    left = v[ j - 1, i - 1] ** 2
    for k in range(0, n):
        if k == i - 1:
            continue
        left *= (lmd[i - 1] - lmd[k])

    pprint("Left side equals to {}".format(left))

    # Now, let's compute right side of the equation (2) in paper

    right = 1
    M = np.delete(A, (j - 1), axis=0)
    M_j = np.delete(M, (j - 1), axis=1)
    lmd_M_j, v_M_j = cal_eigenvalues_and_eigenvectors(M_j)
    for k in range(0, n - 1):
        right *= (lmd[i - 1] - lmd_M_j[k])

    pprint("Right side equals to {}".format(right))

    assert np.abs(left - right) < 1e-5, "left side  {} does not equal to the right side {}.".format(left, right)

In [374]:
check_lemma2()

'Pick a 4 x 4 matrix'
matrix([[0.81544663, 0.79276011, 0.33260111, 0.96847452],
        [0.79276011, 1.26692506, 1.09569135, 0.93080612],
        [0.33260111, 1.09569135, 0.72871825, 0.9783623 ],
        [0.96847452, 0.93080612, 0.9783623 , 0.0549468 ]])
'Lambda Shape : (4,)'
'V Shape: (4, 4)'
'Pick one dimension to check : 3'
'Pick one dimension to delete : 3'
'Left side equals to -0.6200569193511604'
'Right side equals to -0.6200569193511597'


In [410]:
def subidx(n:int, remove:int) -> list:
    out = list(range(n))
    out.remove(remove)
    return out

def submatrix(A:torch.tensor, j:int) -> torch.tensor:
    n = A.shape[0]
    s = subidx(n, j)
    o = A[s, :]
    o = o[:, s]
    return o

def get_eigenvector_val_old(hermitian_matrix, i, j, disable_complex=False):
    # Old way
    lam, v = torch.linalg.eig(hermitian_matrix)
    old_eigenvector_ij = v[j,i]
    
    return old_eigenvector_ij

def get_eigenvector_val(hermitian_matrix, i, j, disable_complex=False):
    # Only need to calculate eigenvalues
    lam  = torch.linalg.eigvals(hermitian_matrix)
    
    n = len(lam)
    M = submatrix(hermitian_matrix, j)
    # Only need to calculate eigenvalues
    lam_submatrix = torch.linalg.eigvals(M)

    # Left side of equation 2
    left = torch.prod(torch.tensor([lam[i] - lam[k] for k in range(n) if k!=i]))
    # Right side of equation 2
    right = torch.prod(torch.tensor([lam[i] - lam_submatrix[k] for k in range(n-1)]))
    # Right divided by left
    eigenvector_ij = right / left

    if disable_complex:
        eigenvector_ij = torch.abs(right)
    
    return eigenvector_ij**0.5

In [411]:
# Random square matrix
rand_square_matrix = torch.rand(50, 50, dtype=float)

# Symmetric matrix: multiply matrix by the transpose of its own
symmetric_matrix = rand_square_matrix * rand_square_matrix.T

# Old 
old_eigenvector_ij = get_eigenvector_val_old(symmetric_matrix, i=0, j=0)

# New
new_eigenvector_ij = get_eigenvector_val(symmetric_matrix, i=0, j=0)

print(f'Old Method Eigenvector ij: {old_eigenvector_ij}')
print('-'*50)
print(f'New Method Eigenvector ij: {new_eigenvector_ij}')

Old Method Eigenvector ij: (-0.1515468685131484+0j)
--------------------------------------------------
New Method Eigenvector ij: (0.1515468685131406+0j)


In [ ]:
     n = np.random.randint(low=3, high=10)  # Dimension of a Hermitian matrix
    C = np.matrix(np.random.rand(n, n))              # Seed Matrix
    A = (C.getH() + C)                         # Construct Hermitian matrix
    pprint("Pick a {} x {} matrix".format(n, n))
    pprint(A)

    lmd, v = cal_eigenvalues_and_eigenvectors(A)
    
    # Now, let's compute left side of equation (2) in paper
    left = v[ j - 1, i - 1] ** 2
    for k in range(0, n):
        if k == i - 1:
            continue
        left *= (lmd[i - 1] - lmd[k])

    pprint("Left side equals to {}".format(left))

    # Now, let's compute right side of the equation (2) in paper

    right = 1
    M = np.delete(A, (j - 1), axis=0)
    M_j = np.delete(M, (j - 1), axis=1)
    lmd_M_j, v_M_j = cal_eigenvalues_and_eigenvectors(M_j)
    for k in range(0, n - 1):
        right *= (lmd[i - 1] - lmd_M_j[k])

In [418]:
# Only need to calculate eigenvalues
lam  = torch.linalg.eigvals(symmetric_matrix)

n = len(lam)
M = submatrix(symmetric_matrix, j)
# Only need to calculate eigenvalues
lam_submatrix = torch.linalg.eigvals(M)

# Left side of equation 2
left = torch.prod(torch.tensor([lam[i] - lam[k] for k in range(n) if k!=i]))
# Right side of equation 2
right = torch.prod(torch.tensor([lam[i] - lam_submatrix[k] for k in range(n-1)]))

In [419]:
M = submatrix(symmetric_matrix, j)

tensor(-5.4464e+19+0.j, dtype=torch.complex128)

In [420]:
right

tensor(-3.6112e+18+0.j, dtype=torch.complex128)